# Leia V3

This version of Leia uses GPT-4 in combination with the function tool to Lookup LeiaActions and Controls.

In [1]:
import os
import json
import sys

## Read and set API keys

In [2]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Logging

In [3]:
# Set up logging
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO) #DEBUG, INFO, WARNING, ERROR, CRITICAL
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# you can set a tokenizer directly, or optionally let it default
# to the same tokenizer that was used previously for token counting
# NOTE: The tokenizer should be a function that takes in text and returns a list of tokens
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode,
    verbose=True  # set to true to see usage printed to the console
    )
callback_manager = CallbackManager([token_counter])

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


## Set up the functions to retrieve Actions and Controls

In [ ]:
# define function tool
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

# hardcode top k for now
top_k = 3

# define vector store info describing schema of vector store
vector_store_info_actions = VectorStoreInfo(
    content_info="collection of actions",
    metadata_info=[
        MetadataInfo(
            name="short_description",
            type="str",
            description="descriptio of what the action does",
        ),
        MetadataInfo(
            name="name",
            type="str",
            description="name of the action",
        ),
    ],
)

# define pydantic model for auto-retrieval function
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        ),
    )


def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)


description = f"""\
Use this tool to look up biographical information about celebrities.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="celebrity_bios",
    description=description,
    fn_schema=AutoRetrieveModel,
)

## Load the index

In [5]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


## GPT 4

In [6]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4,callback_manager=callback_manager)

### create Prompt Template

In [7]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood.\n"
    "In LiquidEarth, a 'Space' and a 'Project are the same thing. We have provided Documentation on the software and further information below. In some cases the metadata includes a 'Control' Field that points to a UI element in the app associated to the described functionality. this is only for internal use. when describing controls to the user, use descriptions and names from the text, not the 'control' values. \n"
    "If the user asks you to do something in LiquidEarth, look for the right 'LeiaAction' in the context and compile the function call based on the information and example. do not use any Actions that are not documented. return the function call at the end of your response in curly braces. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, return the 'Control' properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Append the list to your response without further comment. If no controls are found, do not comment it. Never include any controls that are not specified in the Metadata Field in the provided documentation. Do not interpret any controls from the text body. If the answer requires multiple steps, describe each step in detail. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

In [8]:
query_engine = index.as_query_engine(service_context=service_context_gpt4, text_qa_template=QA_TEMPLATE, retriever_mode="embedding",callback_manager=callback_manager)

In [15]:
#dirty hack: trying to increase the context size
query_engine.retriever._similarity_top_k = 10

In [16]:
response = query_engine.query("please change the Background to Blue")

LLM Prompt Token Usage: 491
LLM Completion Token Usage: 130


In [17]:
print(response)

Sure, I can help you with that. To change the background to blue in LiquidEarth, you need to follow these steps:

1. Open the settings menu in the application.
2. Look for the option labeled 'Background'.
3. In the 'Background' settings, you will find different options for the background of the virtual space.
4. Select the option for a blue background.

Please note that the actual names and descriptions of the options may vary slightly based on the version of the software you are using. 

Here is the function call you need: {LeiaAction: "ChangeBackground", Parameters: {"Color": "Blue"}}.


## Chat with a prompt template (ToDo)

In [ ]:
custom_prompt = Prompt("""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
""")

# list of (human_message, ai_message) tuples
custom_chat_history = [
    (
        'Hello assistant, we are having a insightful discussion about Paul Graham today.',
        'Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True
)

## print token usage

In [12]:
print('Embedding Tokens: ', token_counter.total_embedding_token_count, '\n',
      'LLM Prompt Tokens: ', token_counter.prompt_llm_token_count, '\n',
      'LLM Completion Tokens: ', token_counter.completion_llm_token_count, '\n',
      'Total LLM Token Count: ', token_counter.total_llm_token_count)

Embedding Tokens:  0 
 LLM Prompt Tokens:  1393 
 LLM Completion Tokens:  318 
 Total LLM Token Count:  1711
